In [1]:
import pandas as pd 
import numpy as np
import os
import matplotlib.pyplot as plt
from plotutils import plot_data, plot_surface
from SVM import kernel_svm_train, kernel_svm_predict
from sklearn import preprocessing
from crossval import cross_validate
from RVM import kernel_rvm_train, kernel_rvm_predict, log_reg
from linearclassifier import linear_predict

In [2]:
# Read the data using pandas
df = pd.read_csv('digit-recognition/digit-recognition/train.csv')
df2 = pd.read_csv('digit-recognition/digit-recognition/test.csv')

#turn the training dataframe into numpy array 
label_data_train = df.to_numpy()

#column vector for the labels (38000,)
label_train = label_data_train[:,0] 

# (38000 x 784 matrix for the training data )
data_train = label_data_train[:, 1:] 
print(label_train.shape)
print(data_train.shape)

# Turn testing dataframe into numpy array 
label_data_test = df2.to_numpy()
label_test = label_data_test[:,0]
data_test = label_data_test[:,1:]

masked_train_labels = np.full(label_train.shape, -1)
masked_test_labels = np.full(label_test.shape, -1)

(38000,)
(38000, 784)


In [3]:
c_vals = 10 ** np.linspace(-3, 1, 5)
sigmas = np.linspace(0.1, 1.5, 15)
orders = [2, 3, 4, 5]
norms = ['l1', 'l2', 'max']
num_folds = 4
sets = len(np.unique(label_train))
test_accuracy = np.zeros(sets)

In [4]:
train_indices = np.where(label_train==1)[0]
test_indices = np.where(label_test==1)[0]
masked_train_labels = np.full(label_train.shape, -1)
masked_train_labels[train_indices] = 1
masked_test_labels = np.full(label_test.shape, -1)
masked_test_labels[test_indices] = 1
best_params = {}
best_params['kernel'] = 'polynomial'
best_params['C'] = 1
best_params['sigma'] = .1
best_params['order'] = 100
split_train_data = np.split(data_train, 50)[0]
split_train_labels = np.split(masked_train_labels, 50)[0]
split_train_data_normalized = preprocessing.normalize(split_train_data, norm='l2')
model = kernel_rvm_train(split_train_data_normalized.T, split_train_labels, best_params)
scores = kernel_rvm_predict(split_train_data_normalized.T, model)
print(scores)
#print(scores.shape)
#print(scores) -1's
#print(split_train_labels.shape)
model2 = log_reg(scores, split_train_labels)
test_scores = kernel_rvm_predict(data_test.T, model)
print(data_test.T.shape)
print(test_scores)
results = linear_predict(scores.reshape(1,-1), model2)
for i in range(len(results)):
    print(results[i])
print(results)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [5]:
# Run rbf kernel with no cross validation (random value selection)
for i in range(sets):
    train_indices = np.where(label_train==i)[0]
    test_indices = np.where(label_test==i)[0]

    masked_train_labels = np.full(label_train.shape, -1)
    masked_train_labels[train_indices] = 1

    masked_test_labels = np.full(label_test.shape, -1)
    masked_test_labels[test_indices] = 1

    best_params = {}
    best_params['kernel'] = 'rbf'
    best_params['C'] = 1
    best_params['sigma'] = .1
    
    split_train_data = np.split(data_train, 50)[0]
    print(split_train_data.shape)
    split_train_labels = np.split(masked_train_labels, 50)[0]
    
    split_train_data_normalized = preprocessing.normalize(split_train_data, norm='l2')
    
    model = kernel_svm_train(split_train_data_normalized.T, split_train_labels, best_params)
    predictions = kernel_svm_predict(data_test.T, model)
    test_accuracy[i] = np.mean(predictions[0] == masked_test_labels)
    print("Test accuracy for {0}: {1}".format(i, test_accuracy[i]))
    
print("Overall accuracy: {0}".format(np.mean(test_accuracy))) #this is incorrect

(760, 784)
Test accuracy for 0: 0.9
(760, 784)
Test accuracy for 1: 0.8885
(760, 784)
Test accuracy for 2: 0.8925
(760, 784)
Test accuracy for 3: 0.9075
(760, 784)
Test accuracy for 4: 0.90275
(760, 784)
Test accuracy for 5: 0.90525
(760, 784)
Test accuracy for 6: 0.89675
(760, 784)
Test accuracy for 7: 0.89775
(760, 784)
Test accuracy for 8: 0.90425
(760, 784)
Test accuracy for 9: 0.90475
Overall accuracy: 0.9
